In [1]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
import import_ipynb
#os정렬
import natsort
from tqdm import tqdm
# from standard_func import *
# import standard_func_py as sf
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 진행 여부 함수
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print("실패: 'y' 혹은 'n' 를 입력해 주세요")

# null/입력 값 검색 로직
def nll_chk(data):
    cnt_chk = len(data)

    chk_nll_list = []
    chk_nll_cnt_list = []
    # null값 체크(개수 파악)
    for i,j in zip(range(len(data.columns)), data.columns):
        indv_cnt = data.loc[data[j] != '', j]
        if len(data[j]) != len(indv_cnt):
            print(j + ": " + str(len(indv_cnt)))
            chk_nll_list.append(j)
            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)
    print("================================")
    if len(chk_nll_list) == 0:
        print("모든 값이 입력되었습니다.")
    elif len(chk_nll_list) > 1 :
        print("null이 포함된 컬럼이 탐지되었습니다.")
        print(chk_nll_list)
        print("================================")

    globals()["cnt_chk_out"] = pd.DataFrame(zip(chk_nll_list, chk_nll_cnt_list), columns = ["NULL포함컬럼명", "NULL개수"])
    print(cnt_chk_out)
    
    print("================================")
    print("null 인덱스를 임시 보관합니다.")
    print("================================")
    nll_chk_row = cnt_chk_out.iloc[:,0].to_list()
    for i, j in zip(range(len(nll_chk_row)), nll_chk_row):
        globals()["null_{}".format(i)] = data.loc[data[j] == '', j].index
        print("널인덱스보관_" + str(j) + "(변수명) -> null_{}".format(i))
        
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
#         print(type(col))

# 불필요 컬럼 빼기 - 이름으로 빼기
def brief_col(data, excpt_col_list, name):
    col_list = data.columns
    selected_col_list = [x for x in col_list if x not in excpt_col_list]
    print(selected_col_list)
    globals()["{}".format(name)] = data[selected_col_list]
    
# 불필요 컬럼 빼기 - 순서로 빼기 
# tmp_col_list = [5,6,7,8]
# tmp_tbl_list = [2,3,4,5]
def brief_col_num(data, excpt_col_list, name):
    col_list = data.columns
    col_list_num = [*range(len(col_list))]
    selected_col_list_num = [x for x in col_list_num if x not in excpt_col_list]
    selected_col = col_list[selected_col_list_num]
    print(selected_col)
    globals()["{}".format(name)] = data[selected_col]

# 컬럼명(순서) 선택 및 변경
def cor_col(data, select_col, modify_col, name):
    # 컬럼명(순서) 선택
    globals()["{}".format(name)] = data[select_col]
    # 컬렴명 변경
    globals()["{}".format(name)].columns = modify_col
    
# null 공백처리 함수
def nll_to_blnk(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 Null 공백 처리]")
        for i in data.columns:
            data[i] = data[i].fillna("")
            print(str(i) + ":\t Null 공백 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].fillna("")
        print(col + ":\t  공백 처리 성공")

# 컬럼 고유값 수 비교
def cmpar(data, col1, col2):
    crtr_1 = len(data[col1].unique()) - 1
    crtr_2 = len(data[col2].unique()) - 1
    if crtr_1 == crtr_2:
        print('[정상] 용어명과 분리된용어명의 고유값 수가 같습니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
    else:
        print('[오류] 용어명과 분리된용어명의 고유값 수가 다릅니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
        
# 공백 null처리 함수
def blank_to_nll(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 공백 Null 처리]")
        for i in data.columns:
            data.loc[data[i] == '', i] = None
            print(str(i) + ":\t Null 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data.loc[data[col] == '', col] = None
        print(col + ":\t  Null 처리 성공")
        
def row_nll_rmv(data, name):
    ### 값이 한개라도 있을 경우만 카운트 ###

    data_cnt = len(data)
    notna_list = []
    for i in range(data_cnt):    
        # row 기준 널 체크(값이 한개라도 있을 경우 포함)
        if data.iloc[i].notna().sum() > 1:
            notna_list.append(i)
    globals()["{}".format(name)] = globals()["{}".format(data)].loc[notna_list]
    print(notna_list)
    print(data.shape)
    
# blank row 삭제
def blnk_row_rmv(data, opt, *col):
    if opt == 'all':
        data_cnt = len(data)
        notna_list = []
        for i in range(data_cnt):
            if (data.iloc[i,:] != '').sum() > 1:
                notna_list.append(i)
        globals()["blnk_rmv_all".format(col)] = data.iloc[notna_list]
        print(globals()["blnk_rmv_all".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
    elif opt == 'col':
        data_cnt = len(data)
        notna_list = []
        col = ''.join(col)
        for i in range(data_cnt):
            if (data.loc[i,col] != ''):
                notna_list.append(i)
        globals()["blnk_rmv_{}".format(col)] = data.iloc[notna_list].copy()
        print(globals()["blnk_rmv_{}".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
        
        
def emoti_chk(data, col, rglr_exp, *opt):
    
    # 이력 보기 옴션
    opt = ''.join(opt)
    if opt == 'view':
        stwd_total = data

        # 한글명에 영문 소문자 탐색
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in tqdm(range(len(stwd_total))):

            # 조건에 맞는 단어만 체크
            cor_kor = re.findall(rglr_exp, stwd_total.loc[i, col])
            # 조건에 맞는 단어 수 체크
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, col]))
            
            # 이력 저장
            if bool(chk_kor) == False:
                print("----------------------------- 수정 ----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, col] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 한글 외 변경 수정
            stwd_total.loc[i, 'kor_ec'] = re.sub(rglr_exp, '', stwd_total.loc[i, col])        

            # 한글 외 자동 삭제
            stwd_total.loc[i, col] = ''.join(cor_kor)

#         # 변경 이력 저장
#         stwd_total['kor_cor_hist'] = ""
#         stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("한글 조합어 외 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')
    else:
        
        # 이력 보지 않기 옴션(default)
        stwd_total = data

        # 한글명에 영문 소문자 탐색
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in tqdm(range(len(stwd_total))):

            # 조건에 맞는 단어만 체크
            cor_kor = re.findall(rglr_exp, stwd_total.loc[i, col])
            # 조건에 맞는 단어 수 체크
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, col]))
            
            # 이력 저장
            if bool(chk_kor) == False:
                kor_chk_list.append(i)

            # 한글 외 변경 수정
            stwd_total.loc[i, 'kor_ec'] = re.sub(rglr_exp, '', stwd_total.loc[i, col])        

            # 한글 외 자동 삭제
            stwd_total.loc[i, col] = ''.join(cor_kor)

#         # 변경 이력 저장
#         stwd_total['kor_cor_hist'] = ""
#         stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("한글 조합어 외 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

        

### 1. 전처리(특수기호 제거)

In [2]:
# 불러오기
path = 'C:/Users/namdd/OneDrive/Data/Standardization/dict'
path_dir = 'C:/Users/namdd/OneDrive/Data/Standardization/dict/전체 내려받기_표준국어대사전_xls_20221101'
file_list = os.listdir(path_dir)
# window 정렬기준 사용
file_list = natsort.natsorted(file_list)

In [3]:
# 파일명(xlsx) 제거
file_list_Nxls = []
for i in file_list:
    file_list_Nxls.append(i.split('.')[0])
file_list_Nxls

['1047090_30000',
 '1047090_60000',
 '1047090_90000',
 '1047090_120000',
 '1047090_150000',
 '1047090_180000',
 '1047090_210000',
 '1047090_240000',
 '1047090_270000',
 '1047090_300000',
 '1047090_330000',
 '1047090_360000',
 '1047090_390000',
 '1047090_420000',
 '1047090_434216']

In [4]:
# 모든 파일 읽어오기
for i in tqdm(file_list_Nxls):
    globals()["dict_{}".format(i)] = pd.read_excel(path_dir + "/" + i + ".xls")

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:46<00:00,  3.09s/it]


In [5]:
# 모든 파일 병합
dict_total = pd.DataFrame()
for i in tqdm(file_list_Nxls):
    dict_total = pd.concat([dict_total, globals()["dict_{}".format(i)]], axis = 0, ignore_index = True)
dict_smpl_mrg = dict_total.copy()    
print(dict_total.shape)
dict_total.head(2)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:01<00:00, 11.83it/s]


(434216, 26)


,어휘,구성 단위,고유어 여부,원어,원어·어종,어원,주표제어,부표제어,발음,활용,검색용 이형태,품사,공통 문형,의미 문형,공통 문법,의미 문법,뜻풀이,용례,범주,전문 분야,속담,관용구,대역어,생물 분류군 정보,멀티미디어,관련 어휘
0,가(01),단어,고유어,NaN,NaN,＜＜＜용가＞,NaN,NaN,[가ː],NaN,NaN,「명사」\n,NaN,「1」\n「2」\n「3」\n「4」\n,NaN,「1」\n「2」\n「3」\n「4」((일부 명사 뒤에 붙어))\n,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...,「1」일반어\n「2」일반어\n「3」일반어\n「4」일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,가(02),단어,고유어,NaN,NaN,NaN,NaN,NaN,[가],NaN,NaN,「명사」\n,NaN,\n,NaN,\n,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN,일반어\n,『음악』\n,NaN,NaN,NaN,NaN,NaN,"의미\n동의어(2) : 아08(A), 에이05(A / a)\n\n"


In [6]:
# 모든 품사COL 특수기호 제거
emoti_chk(dict_total, '품사', "[가-힣+]")

100%|████████████████████████████████████████████████████████████████████████| 434216/434216 [01:53<00:00, 3842.06it/s]


====================================한글명 검토====================================|
한글 조합어 외 제거: 434216
[자동 처리 완료하였습니다]


In [7]:
pumsa_unique = dict_total['품사'].unique()
pumsa_unique

array(['명사', '품사없음', '조사', '접사', '동사', '형용사', '명사부사', '부사', '구', '명사관형사',
       '부사명사', '동사보조동사', '부사감탄사', '의존명사', '동사형용사', '감탄사', '명사의존명사', '관형사',
       '수사관형사', '형용사동사', '부사조사', '의존명사대명사', '대명사', '어미', '대명사감탄사',
       '명사감탄사', '부사품사없음', '명사대명사', '대명사관형사', '관형사품사없음', '감탄사품사없음',
       '감탄사명사', '명사수사', '명사품사없음', '명사수사관형사', '관형사감탄사', '대명사명사', '수사',
       '수사관형사명사', '의존명사명사', '어미품사없음', '보조형용사', '보조동사', '의존명사조사',
       '명사수사관형사부사', '의존명사수사관형사', '동사형용사보조동사보조형용사', '명사부사감탄사',
       '동사보조동사보조형용사', '품사없음명사', '명사관형사부사', '명사의존명사대명사', '보조동사보조형용사',
       '감탄사부사', '부사감탄사품사없음', '대명사부사', '동사형용사보조동사', '동사보조형용사', '수사관형사명사부사',
       '수사명사'], dtype=object)

In [8]:
# 모든 어휘COL 특수기호 제거
emoti_chk(dict_total, '어휘', "[가-힣+]")

100%|████████████████████████████████████████████████████████████████████████| 434216/434216 [01:44<00:00, 4161.59it/s]


====================================한글명 검토====================================|
한글 조합어 외 제거: 372956
[자동 처리 완료하였습니다]


In [9]:
#결론:품사==명사는 구성단위=단어 조건을 포함한다 

# dict_total.loc[(dict_total['구성 단위'] == '단어') & (dict_total['품사'] == '명사')].shape
dict_total.loc[(dict_total['품사'] == '명사')].shape

(266276, 28)

In [10]:
eohee_unique = dict_total['어휘'].unique()
eohee_unique

array(['가', '가가', '가가대소', ..., '딘다', '받침체언', '종성체언'], dtype=object)

### 2. 품사 및 예시 데이터 생성

In [11]:
# 품사 기준 50개 데이터 샘플
dict_clssf = dict_total.columns
pumsa_examp = pd.DataFrame()
select_cnt = 50
for i, j in tqdm(zip(range(len(pumsa_unique)), pumsa_unique)):

    row_cnt = len(dict_total.loc[dict_total['품사'] == j, '어휘'])
    globals()["pumsa_unique_{}".format(i)] = pd.DataFrame(
        dict_total.loc[dict_total['품사'] == j, '어휘'].head(select_cnt).reset_index(drop=True))
    globals()["pumsa_unique_{}".format(i)].columns = [j]
    globals()["pumsa_unique_{}".format(i)].loc[select_cnt+1,j] = row_cnt
    pumsa_examp = pd.concat([pumsa_examp, globals()["pumsa_unique_{}".format(i)]], axis = 1)
print(pumsa_examp.shape)
pumsa_examp.tail(5)

60it [00:04, 13.28it/s]

(51, 60)


,명사,품사없음,조사,접사,동사,형용사,명사부사,부사,구,명사관형사,부사명사,동사보조동사,부사감탄사,의존명사,동사형용사,감탄사,명사의존명사,관형사,수사관형사,형용사동사,부사조사,의존명사대명사,대명사,어미,대명사감탄사,명사감탄사,부사품사없음,명사대명사,대명사관형사,관형사품사없음,감탄사품사없음,감탄사명사,명사수사,명사품사없음,명사수사관형사,관형사감탄사,대명사명사,수사,수사관형사명사,의존명사명사,어미품사없음,보조형용사,보조동사,의존명사조사,명사수사관형사부사,의존명사수사관형사,동사형용사보조동사보조형용사,명사부사감탄사,동사보조동사보조형용사,품사없음명사,명사관형사부사,명사의존명사대명사,보조동사보조형용사,감탄사부사,부사감탄사품사없음,대명사부사,동사형용사보조동사,동사보조형용사,수사관형사명사부사,수사명사
46,가거,가견,란,관,가납되다,가란하다,내일모레,가뜩이나,가닥이잡히다,거시적,NaN,NaN,NaN,국제옴,기연미연하다,둥개둥개,바람,무슨,삼,NaN,NaN,NaN,그분,기로,NaN,어깨총,NaN,소사,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,려거든,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,가거,가경,랑,관,가납되다,가랄하다,낼,가뜩한데,가도오도못하다,거인적,NaN,NaN,NaN,국제옹스트롬,기우뚱기우뚱하다,둥둥,바이트,뭇,삼십,NaN,NaN,NaN,그이,기로니,NaN,어부바,NaN,소인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,련,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,가거지지,가계경제,로,관,가납하다,가랑가랑하다,낼모레,가뜩히,가락을떼다,거족적,NaN,NaN,NaN,국제와트,기우뚱하다,둬둬,바퀴,뭔,서너째,NaN,NaN,NaN,그자,기로서,NaN,어비,NaN,소자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,렵니까,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,가거처,가계미가,로부터,광,가납하다,가량가량하다,냠냠,가뜬가뜬,가락이나다,거족적,NaN,NaN,NaN,군데,기웃기웃하다,둬둬둬,박,바른,서른,NaN,NaN,NaN,그쪽,기로서니,NaN,엎드려뻗쳐,NaN,소제,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,렵니다,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,266276,70520,173,533,55503,12721,300,11912,11308,1229,19,27,19,720,625,503,132,138,123,24,2,4,251,729,6,80,23,75,13,7,9,9,1,9,12,12,1,18,16,4,77,11,6,4,1,4,1,4,3,5,1,1,2,2,1,1,1,1,1,3


In [12]:
# dict_smpl_mrg.to_excel(path + '/dict_smpl_mrg/' + 'dict_smpl_mrg.xlsx')
# dict_total.to_excel(path + '/dict_total/' + 'dict_total.xlsx')
dict_total.to_csv(path + '/dict_total/' + 'dict_total.csv')

In [13]:
pumsa_examp.to_excel(path + '/pumsa_examp/' + 'pumsa_examp.xlsx')

20만단어 초반대인데... expln 입히려면 시간 좀 걸리는게 맞고...... \n
20만단어를 가지고 표준단어사전을 점검하는게 과연 가능할지.......

In [14]:
dict_total[dict_total['어휘'] == '창고']

,어휘,구성 단위,고유어 여부,원어,원어·어종,어원,주표제어,부표제어,발음,활용,검색용 이형태,품사,공통 문형,의미 문형,공통 문법,의미 문법,뜻풀이,용례,범주,전문 분야,속담,관용구,대역어,생물 분류군 정보,멀티미디어,관련 어휘,kor_lower,kor_ec
357778,창고,단어,한자어,倉庫,한자 倉庫\n,NaN,NaN,NaN,[창고],NaN,NaN,명사,NaN,\n,NaN,\n,물건이나 자재를 저장하거나 보관하는 건물.\n,창고에 곡식이 산더미처럼 쌓여 있었다.\n창고에 처박혀 있던 고문서 더미를 찾아냈다...,일반어\n,NaN,NaN,NaN,NaN,NaN,사진-창고\n사진-창고\n사진-창고\n,의미\n비슷한말(1) : 곳집「1」(庫집)\n\n,,(01)
357779,창고,단어,한자어,蒼古,한자 蒼古\n,NaN,NaN,창고-하다,NaN,NaN,NaN,명사품사없음,NaN,[Ⅰ]\n[Ⅱ]\n,NaN,[Ⅰ]\n[Ⅱ]\n,[Ⅰ]아주 먼 옛 시대.\n[Ⅱ]‘창고하다01’의 어근.\n,NaN,[Ⅰ]일반어\n[Ⅱ]일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,(02)
357780,창고,단어,한자어,蒼枯,한자 蒼枯\n,NaN,NaN,창고-하다,NaN,NaN,NaN,품사없음,NaN,\n,NaN,\n,‘창고하다02’의 어근.\n,NaN,일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,(03)


### 3. dict_insp 생성: 조사를 위한 사전(uniqueList) csv 생성

In [15]:
pum_list = ['명사', '품사없음', '의존명사', '명사의존명사', '수사관형사', '명사대명사', '수사', '수사관형사명사', '의존명사명사']
dict_insp = pd.DataFrame(dict_total.loc [dict_total['품사'].isin(pum_list), '어휘'].unique(), columns= ['어휘'])
print(dict_insp.shape)
dict_insp

(278234, 1)


,어휘
0,가
1,가가
2,가가대소
3,가가례
4,가가린
...,...
278229,자형자물쇠
278230,라오
278231,려우
278232,받침체언


In [16]:
dict_insp[dict_insp['어휘'] == '등급']

,어휘
66366,등급


In [17]:
dict_insp.to_csv(path + '/dict_total/' + 'dict_insp.csv')
print('생성 테스트 결과: ')
pd.read_csv(path + '/dict_total/' + 'dict_insp.csv', index_col=0).head()

생성 테스트 결과: 


,어휘
0,가
1,가가
2,가가대소
3,가가례
4,가가린


### 4. dict_expn 생성: 한자+설명

In [18]:
dict_expn = dict_total.loc [dict_total['품사'].isin(pum_list) , ['원어', '어휘', '뜻풀이', '용례']]
print(dict_expn.shape)
dict_expn

(337884, 4)


,원어,어휘,뜻풀이,용례
0,NaN,가,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...
1,NaN,가,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN
2,加,가,‘가하다01’의 어근.\n,NaN
3,加,가,"부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리를 뜻...",NaN
4,可,가,"「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 찬성하...","「1」이 사람 말도 가요, 저 사람 말도 가요 하면 도대체 어떤 사람 말을 따라야 ..."
...,...,...,...,...
434211,NaN,,"한글 자모 ‘ㅜ’와 ‘ㅣ’를 어울러 쓴 글자. ‘위’라고 이르며, ‘귀’에서와 같이...",NaN
434212,NaN,,"한글 자모의 스물두째 글자. ‘유’라고 이르며, ‘규, 육’에서와 같이 중성으로 쓰...",NaN
434213,NaN,,"한글 자모의 스물셋째 글자. ‘으’라고 이르며, ‘그, 극’에서와 같이 중성으로 쓰...",NaN
434214,NaN,,"한글 자모 ‘ㅡ’와 ‘ㅣ’를 어울러 쓴 글자. ‘의’라고 이르며, ‘의’에서와 같이...",NaN


In [19]:
nll_to_blnk(dict_expn, 'all')

[모든 컬럼 Null 공백 처리]
원어:	 Null 공백 처리 성공
어휘:	 Null 공백 처리 성공
뜻풀이:	 Null 공백 처리 성공
용례:	 Null 공백 처리 성공


In [20]:
dict_expn.to_csv(path + '/dict_total/' + 'dict_expn.csv')
print('생성 테스트 결과: ')
pd.read_csv(path + '/dict_total/' + 'dict_expn.csv', index_col=0).head()

생성 테스트 결과: 


,원어,어휘,뜻풀이,용례
0,NaN,가,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...
1,NaN,가,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",NaN
2,加,가,‘가하다01’의 어근.\n,NaN
3,加,가,"부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리를 뜻...",NaN
4,可,가,"「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 찬성하...","「1」이 사람 말도 가요, 저 사람 말도 가요 하면 도대체 어떤 사람 말을 따라야 ..."


In [21]:
# () 삭제
dict_expn['의미'] = dict_expn['원어'] + ". " + dict_expn['뜻풀이']
for i in tqdm(dict_expn.index):
    if dict_expn['의미'][i][0:2] == '. ':
        dict_expn['의미'][i] = dict_expn['의미'][i][2:]
        
# 다른 방법(느림)
# for i in tqdm(dict_expn.index):
#     if dict_expn.loc[i, '원어'] == '':
#         dict_expn.loc[i, 'test'] = dict_expn.loc[i, '뜻풀이']
#     else:
#         dict_expn.loc[i, 'test'] = "(" + dict_expn.loc[i, '원어'] + ")" + dict_expn.loc[i, '뜻풀이']

100%|███████████████████████████████████████████████████████████████████████| 337884/337884 [00:14<00:00, 23751.37it/s]


In [22]:
dict_expn['어휘'].unique()

array(['가', '가가', '가가대소', ..., '려우', '받침체언', '종성체언'], dtype=object)

In [23]:
# 중복 리스트 전부 뽑아내기
dict_expn_dup_list = dict_expn.duplicated(['어휘'],keep=False)
# 뜻풀이 검색 불가(중복) 단어 검색 = left Join
dict_expn_dup = dict_expn[dict_expn_dup_list]
# 뜻풀이 검색 가능(유니크) 단어 검색 = left Join
dict_expn_unq = dict_expn[dict_expn_dup_list==False]

In [24]:
dict_expn_unq.to_excel(path + '/dict_total/' + 'dict_expn_unq.xlsx')
dict_expn_dup.to_excel(path + '/dict_total/' + 'dict_expn_dup.xlsx')

dict_expn_dup.to_csv(path + '/dict_total/' + 'dict_expn_dup.csv')
dict_expn_unq.to_csv(path + '/dict_total/' + 'dict_expn_unq.csv')

In [25]:
dict_expn_dup.head()

,원어,어휘,뜻풀이,용례,의미
0,,가,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...,「3」참기름을 따를 때 가에 흘리지 않도록 조심해라.\n「4」강가.\n「4」냇가.\...,「1」경계에 가까운 바깥쪽 부분.\n「2」어떤 중심 되는 곳에서 가까운 부분.\n「...
1,,가,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n",,"서양 음악의 칠음 체계에서, 여섯 번째 음이름.\n"
2,加,가,‘가하다01’의 어근.\n,,加. ‘가하다01’의 어근.\n
3,加,가,"부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리를 뜻...",,"加. 부여와 고구려에서, 족장이나 고관을 이르던 말. 본디 씨족이나 부족의 우두머리..."
4,可,가,"「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 찬성하...","「1」이 사람 말도 가요, 저 사람 말도 가요 하면 도대체 어떤 사람 말을 따라야 ...","可. 「1」옳거나 좋음.\n「2」회의 따위에서, 어떤 안건에 대하여 표결을 할 때 ..."


In [26]:
dict_expn_unq.head()

,원어,어휘,뜻풀이,용례,의미
21,呵呵大笑,가가대소,소리를 내어 크게 웃음.\n,"위원장은 무엇이 그렇게 우스운지, 세수를 마칠 생각은 않고, 들창코를 벌름거리며 가...",呵呵大笑. 소리를 내어 크게 웃음.\n
23,家家禮,가가례,각 집안에 따라 달리 행하는 예법ㆍ풍속 따위.\n,우리나라 제사 음식 진설은 가가례라 하여 각 가정이나 지역에 따라 조금씩 다르다.\n,家家禮. 각 집안에 따라 달리 행하는 예법ㆍ풍속 따위.\n
24,"Gagarin, Yury Alekseevich",가가린,소련의 군인ㆍ우주 비행사(1934~1968). 1961년 인공위성인 보스토크 1호를...,,"Gagarin, Yury Alekseevich. 소련의 군인ㆍ우주 비행사(1934~..."
25,家家門前,가가문전,집집마다의 문 앞.\n,거지 하나가 가가문전을 찾아다니며 구걸을 했다.\n,家家門前. 집집마다의 문 앞.\n
26,假家房,가가방,‘가겟방’의 원말.\n,가가방에는 불이 켜지고 두런두런 얘기하는 소리가 들린다.\n,假家房. ‘가겟방’의 원말.\n


#### <옵션1> 사전에서 1글자 이상만 대입 조건

In [27]:
def excpt_one(data, col, name): 
    # 1글자 이상 찾기
    more_list = []
    one_list = []
    for i in tqdm(range(len(data[col]))):
        if len(data.iloc[i,0]) > 1:
            more_list.append(i)
        elif len(data.iloc[i,0]) == 1:
            one_list.append(i)
    globals()[name] = data.iloc[more_list]


In [28]:
excpt_one(dict_insp, '어휘', 'dict_expn_oneMore')
print(dict_expn_oneMore.shape)
dict_expn_oneMore

100%|███████████████████████████████████████████████████████████████████████| 278234/278234 [00:07<00:00, 35770.74it/s]


(277376, 1)


,어휘
1,가가
2,가가대소
3,가가례
4,가가린
5,가가문전
...,...
278229,자형자물쇠
278230,라오
278231,려우
278232,받침체언


In [29]:
dict_expn_oneMore['어휘']

1            가가
2          가가대소
3           가가례
4           가가린
5          가가문전
          ...  
278229    자형자물쇠
278230       라오
278231       려우
278232     받침체언
278233     종성체언
Name: 어휘, Length: 277376, dtype: object

In [30]:
dict_expn_oneMore.to_csv(path + '/dict_total/' + 'dict_expn_oneMore.csv')

In [31]:
# 테스트를 위한 검색 1

inp = '관형사품사없음'
inp = '대명사명사'
dict_total.loc[dict_total['품사'] == inp]

,어휘,구성 단위,고유어 여부,원어,원어·어종,어원,주표제어,부표제어,발음,활용,검색용 이형태,품사,공통 문형,의미 문형,공통 문법,의미 문법,뜻풀이,용례,범주,전문 분야,속담,관용구,대역어,생물 분류군 정보,멀티미디어,관련 어휘,kor_lower,kor_ec
64674,나,단어,고유어,NaN,NaN,나＜석상＞,NaN,NaN,[나],NaN,NaN,대명사명사,NaN,[Ⅰ]\n[Ⅱ]「1」\n[Ⅱ]「2」\n,NaN,[Ⅰ]\n[Ⅱ]「1」\n[Ⅱ]「2」\n,[Ⅰ]말하는 이가 대등한 관계에 있는 사람이나 아랫사람을 상대하여 자기를 가리키는 ...,[Ⅰ]나의 소망.\n[Ⅰ]나는 학생이다.\n[Ⅰ]나와 그 친구는 사이가 참 좋다.\...,[Ⅰ]일반어\n[Ⅱ]「1」일반어\n[Ⅱ]「2」일반어\n,[Ⅱ]「2」『철학』\n,· 나 가는 데 강철이 가는 데\n · 나는 바담 풍(風) 해도 너는 바람 풍 해...,· 나 몰라라 하다\n · 나 죽었소 하다\n,NaN,NaN,NaN,의미\n동의어(1) : [Ⅱ]「2」 자아「2」(自我)\n\n,,(03)


In [32]:
# 테스트를 위한 검색 2

inp = '가경'
dict_total.loc[dict_total['어휘'] == inp]

,어휘,구성 단위,고유어 여부,원어,원어·어종,어원,주표제어,부표제어,발음,활용,검색용 이형태,품사,공통 문형,의미 문형,공통 문법,의미 문법,뜻풀이,용례,범주,전문 분야,속담,관용구,대역어,생물 분류군 정보,멀티미디어,관련 어휘,kor_lower,kor_ec
152,가경,단어,한자어,加敬,한자 加敬\n,NaN,NaN,NaN,[가경],NaN,NaN,명사,NaN,\n,NaN,\n,더욱 공경함.\n,NaN,일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,(01)
153,가경,단어,한자어,可驚,한자 可驚\n,NaN,NaN,가경-하다,NaN,NaN,NaN,품사없음,NaN,\n,NaN,\n,‘가경하다’의 어근.\n,NaN,일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,(02)
154,가경,단어,한자어,佳景,한자 佳景\n,NaN,NaN,NaN,[가ː경],NaN,NaN,명사,NaN,\n,NaN,\n,빼어나게 아름다운 경치.\n,동해의 일출 광경은 참으로 가경이다.\n,일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,어휘\n참고 어휘(1) : 절경01(絕景)\n\n의미\n동의어(1) : 수기06...,,(03)
155,가경,단어,한자어,佳境,한자 佳境\n,NaN,NaN,NaN,[가ː경],NaN,NaN,명사,NaN,「1」\n「2」\n,NaN,「1」\n「2」\n,「1」한창 재미있는 판이나 고비.\n「2」경치가 좋은 곳.\n,「1」이야기는 점점 가경으로 접어들었다.\n「1」이장자가 이렇게 생각을 했을 때 장...,「1」일반어\n「2」일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,의미\n동의어(1) : 「2」 묘경「2」(妙境)\n비슷한말(1) : 「1」 자경08...,,(04)
156,가경,단어,한자어,家慶,한자 家慶\n,NaN,NaN,NaN,[가경],NaN,NaN,명사,NaN,\n,NaN,\n,집안의 경사.\n,NaN,일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,(05)
157,가경,단어,한자어,嘉慶,한자 嘉慶\n,NaN,NaN,NaN,[가경],NaN,NaN,명사,NaN,\n,NaN,\n,즐겁고 경사스러움.\n,NaN,일반어\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,(06)
158,가경,단어,한자어,嘉慶,한자 嘉慶\n,NaN,NaN,NaN,[가경],NaN,NaN,명사,NaN,\n,NaN,\n,중국 청나라 인종 때의 연호(1796~1820).\n,NaN,일반어\n,『역사』\n,NaN,NaN,NaN,NaN,NaN,NaN,,(07)
159,가경,단어,한자어,歌磬,한자 歌磬\n,NaN,NaN,NaN,[가경],NaN,NaN,명사,NaN,\n,NaN,\n,"조선 세종 때에, ‘특경’을 이르던 말.\n",NaN,일반어\n,『음악』\n,NaN,NaN,NaN,NaN,NaN,NaN,,(08)
